Inspiration for Neural Net: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

# Data Pre-Processing

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pitcher
from random import shuffle
torch.manual_seed(1)

In [2]:
player = 'verlander'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = pitcher.get_reps(ABs)
reps = pitcher.drop_nas(reps)
reps = pitcher.drop_pitches(reps)

/home/zac/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [3]:
shuffle(reps)

In [10]:
reps[4]

([[0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-1.0624, 2.0866, 93.2, 0.1605, 4.4675, 0, 0, 0, 1]],
 ['NA', 'NA', 'FF'],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 'CU')

In [39]:
for rep in reps:
    prevs,types,prepitch,ptype = rep
    if ptype != 'FF':
        rep = (prevs,types,prepitch,'NF')

In [34]:
player = 'verlander'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = []
for AB in ABs:
    prev_0 = [0]*9
    ptypes = []
    for pitch in AB:
        if pitch[1] != 'FF':
            ptypes.append('NF') 
        else:
            ptypes.append('FF')
    
    prev_pitch_cont = [pitch[2:7].tolist() for pitch in AB]
    #prev_pitch_disc = [pitch[22:38].tolist() for pitch in AB]
    ptypes_ = [pitch[38:42].tolist() for pitch in AB]
    #pitches = zip(prev_pitch_cont,prev_pitch_disc)
    pitches = zip(prev_pitch_cont,ptypes_)
    #prevs = [cont + disc for cont,disc in pitches]
    #pitches_ = zip(prevs,ptypes_)
    prev_pitches = [prev + ptype for prev,ptype in pitches]
    #prev_pitches_ = [prev + ptype for prev,ptype in pitches_]
    pre_pitch = [pitch[7:22].tolist() for pitch in AB]
    #prev_p = zip(prev_pitches_,pre_pitch)
    #prev_pitches = [prev + prepitch for prev,prepitch in prev_p]
    AB_len = len(AB)
    for i in range(AB_len):
        if i == 0:
            prevs = [prev_0]*3
            prev_ptypes = ['NA']*3
        elif i == 1:
            prevs = [prev_0]*2 + [prev_pitches[i-1]]
            prev_ptypes =  ['NA']*2 + [ptypes[i-1]]
        elif i == 2:
            prevs = [prev_0] + prev_pitches[i-2:i]
            prev_ptypes =  ['NA'] + [ptypes[i-2:i]]
        else:
            prevs = prev_pitches[i-3:i]
            prev_ptypes =  ptypes[i-3:i]
        reps.append((prevs,prev_ptypes,pre_pitch[i],ptypes[i]))
reps = pitcher.drop_nas(reps)

In [10]:
#run for less fastballs
train = pitcher.drop_ff(reps[1:30000])
test = reps[30000:]

In [35]:
len(data)

39714

In [37]:
len(reps)*0.8

31581.600000000002

In [38]:
#run for unbiased distribution
train = reps[1:31000]
test = reps[31000:]

In [39]:
# work-in-progress
def get_batches(train,batch_size):   
    out = []
    i = 0
    batch_ptypes = []
    batch_pre_pitch = []
    batch_prev_pitches = []
    for rep in train:
        prev_pitches,prev_types,pre_pitch,ptype = rep
        if i % batch_size == 0:
            out.append([batch_prev_pitches,batch_pre_pitch,batch_ptypes])
            batch_ptypes = [ptype]
            batch_pre_pitch = [pre_pitch]
            batch_prev_pitches = [prev_pitches]
        else:
            batch_ptypes.append(ptype)
            batch_pre_pitch.append(pre_pitch)
            batch_prev_pitches.append(prev_pitches)
        i += 1
    return out[1:]

In [40]:
train_batches = get_batches(train,10)
test_batches = get_batches(test,10)

In [47]:
def test_accuracy(batches,model):
    length = len(batches)*len(batches[0][0])
    num_right = 0
    ch_count = 0
    predict_ch = 0
    cu_count = 0
    predict_cu = 0
    sl_count = 0
    predict_sl = 0
    ff_count = 0
    predict_ff = 0
    ff_right = 0
    ch_right = 0
    sl_right = 0
    cu_right = 0
    for batch in batches:
        with torch.no_grad():
            prev_pitches,pre_pitch, ptypes = batch
            prevs_in = torch.tensor(prev_pitches, dtype=torch.float)
            rep_in = torch.tensor(pre_pitch, dtype=torch.float)
            #targets = tag_to_ix[ptypes]
            if len(ptypes) > 1:
                targets = [tag_to_ix[ptype] for ptype in ptypes]
            else:
                targets = tag_to_ix[ptypes[0]]
            if torch.cuda.is_available():
                tag_scores = model((prevs_in,rep_in)).cuda()
            else:
                tag_scores = model((prevs_in,rep_in))
            #print(tag_scores)
            preds = [tag_score.max(0) for tag_score in tag_scores]
            #print(index.item())
            if len(ptypes) > 1:
                for pred,target in zip(preds,targets):
                    _,index = pred
                    if target == 1:
                        ff_count += 1
                    if index.item() == 1:
                        predict_ff += 1
                    if target == 1 and index.item() == 1:
                        ff_right += 1
                    if target == 0:
                        cu_count += 1
                    if index.item() == 0:
                        predict_cu += 1
                    if target == 0 and index.item() == 0:
                        cu_right += 1
                    if target == 2:
                        sl_count += 1
                    if index.item() == 2:
                        predict_sl += 1
                    if target == 2 and index.item() == 2:
                        sl_right += 1
                    if target == 3:
                        ch_count += 1
                    if index.item() == 3:
                        predict_ch += 1
                    if target == 3 and index.item() == 3:
                        ch_right += 1
                    if index.item() == target:
                        num_right += 1
            else:
                _,index = preds[0]
                if targets == 1:
                    ff_count += 1
                if index.item() == 1:
                    predict_ff += 1
                if targets == 1 and index.item() == 1:
                    ff_right += 1
                if targets == 0:
                    cu_count += 1
                if index.item() == 0:
                    predict_cu += 1
                if targets == 0 and index.item() == 0:
                    cu_right += 1
                if targets == 2:
                    sl_count += 1
                if index.item() == 2:
                    predict_sl += 1
                if targets == 2 and index.item() == 2:
                    sl_right += 1
                if targets == 3:
                    ch_count += 1
                if index.item() == 3:
                    predict_ch += 1
                if targets == 3 and index.item() == 3:
                    ch_right += 1
                if index.item() == targets:
                    num_right += 1
    ff_rate = ff_count/length
    pred_ff_rate = predict_ff/length
    ff_acc = ff_right/ff_count
    cu_rate = cu_count/length
    pred_cu_rate = predict_cu/length
    cu_acc = cu_right/cu_count
    #sl_rate = sl_count/length
    #pred_sl_rate = predict_sl/length
    #sl_acc = sl_right/sl_count
    #ch_rate = ch_count/length
    #pred_ch_rate = predict_ch/length
    #ch_acc = ch_right/ch_count
    accuracy = num_right/length
    print("______________________________________")
    print("curve rate:",cu_rate)
    print("Predicted curve rate:",pred_cu_rate)
    print("curve accuracy:",cu_acc)
    print("______________________________________")
    print("Fourseam Fastball rate:",ff_rate)
    print("Predicted Fourseam Fastball rate:",pred_ff_rate)
    print("Fourseam Fastball accuracy:",ff_acc)
    #print("______________________________________")
    #print("Changeup rate:",ch_rate)
    #print("Predicted changeup rate:",pred_ch_rate)
    #print("changeup accuracy",ch_acc)
    #print("______________________________________")
    #print("slider rate:",sl_rate)
    #print("Predicted slider rate:",pred_sl_rate)
    #print("slider accuracy:",sl_acc)
    print("______________________________________")
    print("Accuracy:",accuracy)
    print("Accuracy above naive guess:",accuracy - ff_rate)
    print("______________________________________")
    

# Model Construction

In [57]:
#change this for different pitcher
'''tag_to_ix = {'CU':0,
             'FF':1,
             'SL':2,
             'CH':3}'''
tag_to_ix = {'NF':0,
             'FF':1}

# -- Input Dimensions -- DO NOT CHANGE
PREV_PITCH_DIM = 9
NUM_PREV_PITCHES = 3
GAME_STATE_DIM = 15 
GAME_OUT_DIM = 15

# -- Hyperparameters -- DO CHANGE 
HIDDEN_DIM = 20
OUT_DIM = 15
batch_size = 10
lstm_layers = 1
learning_rate = 0.001

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    
class PitchPredict(nn.Module):
    def __init__(self, prev_pitch_dim, hidden_dim, num_prev_pitches,out_dim, game_state_dim, game_out_dim, num_ptypes):
        super(PitchPredict, self).__init__()
        #get constants
        self.hidden_dim = hidden_dim
        self.prev_pitch_dim = prev_pitch_dim
        self.num_prev_pitches = num_prev_pitches
        self.out_dim = out_dim
        self.game_state_dim = game_state_dim
        self.game_out_dim = game_out_dim
        
        ####Define Layers####
        
        ########################## LSTM for past five pitches#######################################
        # The LSTM takes previous pitch vectors as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(prev_pitch_dim, hidden_dim,num_layers=lstm_layers,batch_first=True)#FIDDLE WITH num_layers

        # The linear layer that maps from hidden state space to a vector
        # with dimensionality OUT_DIM
        self.hidden2out = nn.Linear(hidden_dim, out_dim)
        
        ############## FULLY CONNECTED LAYERS for LSTM OUTPUT + game_state vector #################
        
        # layer to map the game state vector to a different dimension
        self.l1 = nn.Linear(self.game_state_dim, self.game_out_dim)  
        
        # This Fully connected layer maps from the output of the final hidden layer output from the LSTM,
        # dimension = OUT_DIM, with the game state vector, dimension = GAME_STATE_DIM
        # to a vector of length of the number of ptypes to pass through softmax for probabilities
        self.fc1 = nn.Linear((self.out_dim + self.game_out_dim), num_ptypes)

                             
    def forward(self, rep):
        past_pitches,game_state = rep
        lstm_out, _ = self.lstm(past_pitches.view(batch_size,self.num_prev_pitches, -1))
        learned_rep = self.hidden2out(lstm_out.view(batch_size,self.num_prev_pitches, -1))
        encoding = F.relu(learned_rep[:,self.num_prev_pitches - 1:,:])
        game_rep = game_state.view(batch_size,self.game_state_dim)
        #game_rep = self.l1(game_state.view(batch_size,self.game_state_dim))
        #game_rep = F.relu(game_rep)
        #encoding = learned_rep[:,self.num_prev_pitches - 1:,:]
        fc_in = torch.cat((encoding.view(batch_size,self.out_dim),game_rep.view(batch_size,self.game_out_dim)),dim=1)
        fc = self.fc1(fc_in.view(batch_size,self.game_out_dim+self.out_dim))
        tag_scores = F.log_softmax(fc,dim=0)
        return tag_scores

# Training and Testing

In [58]:
model = PitchPredict(PREV_PITCH_DIM, HIDDEN_DIM, NUM_PREV_PITCHES, OUT_DIM, GAME_STATE_DIM, GAME_OUT_DIM, len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)#FIDDLE WITH LEARNING RATE (lr)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    print("***************Pre-Training Accuracy*****************")
    test_accuracy(train_batches,model)
print("**************************Training*****************************")
for epoch in range(5):
    #shuffle(train_batches)
    for batch in train_batches:
        prev_pitches,pre_pitch, ptypes = batch
        
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        #get input tensors ready
        prevs_in = torch.tensor(prev_pitches, dtype=torch.float)
        game_state_in = torch.tensor(pre_pitch, dtype=torch.float)
        
        #get target value
        #target = tag_to_ix[ptype]
        target = [ tag_to_ix[ptype] for ptype in ptypes]

        # Step 3. Run our forward pass.
        if torch.cuda.is_available():
            tag_scores = model((prevs_in,game_state_in)).cuda()
        else:
            tag_scores = model((prevs_in,game_state_in))
               
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores.view(batch_size,-1), torch.tensor(target,dtype=torch.long))
        loss.backward()
        optimizer.step()
        
        preds = [tag_score.max(0) for tag_score in tag_scores]

                
    #display post-epoch results
    print('epoch:',epoch+1,"loss:",loss.item())

# See what the scores are after training
with torch.no_grad():
    print("****************Post-Training Accuracy********************")
    test_accuracy(train_batches,model.eval())
    print("**********************************************************")

***************Pre-Training Accuracy*****************
______________________________________
curve rate: 0.4208777024846725
Predicted curve rate: 0.5209422394320748
curve accuracy: 0.5326995323161849
______________________________________
Fourseam Fastball rate: 0.5791222975153275
Predicted Fourseam Fastball rate: 0.47905776056792515
Fourseam Fastball accuracy: 0.487602384799688
______________________________________
Accuracy: 0.5065827686350436
Accuracy above naive guess: -0.07253952888028392
______________________________________
**************************Training*****************************
epoch: 1 loss: 2.3269102573394775
epoch: 2 loss: 2.319260358810425
epoch: 3 loss: 2.304548501968384
epoch: 4 loss: 2.301831007003784
epoch: 5 loss: 2.30079984664917
****************Post-Training Accuracy********************
______________________________________
curve rate: 0.4208777024846725
Predicted curve rate: 0.5510164569215876
curve accuracy: 0.6880318945027984
____________________________

In [59]:
with torch.no_grad():
    print("****************Test Accuracy********************")
    test_accuracy(test_batches,model)
    print("**********************************************************")

****************Test Accuracy********************
______________________________________
curve rate: 0.412396694214876
Predicted curve rate: 0.5447461629279811
curve accuracy: 0.6839393071858002
______________________________________
Fourseam Fastball rate: 0.587603305785124
Predicted Fourseam Fastball rate: 0.4552538370720189
Fourseam Fastball accuracy: 0.5529435402853125
______________________________________
Accuracy: 0.6069657615112161
Accuracy above naive guess: 0.019362455726092076
______________________________________
**********************************************************


### high score using balanced set (25% FF)
- **TRAINING ACCURACY HIGH SCORE:** 38 (39% FF)

- **TEST ACCURACY HIGH SCORE:** 43.7 (37% FF) -14%- naive 


- ****HYPERPARAMETERS:****

    - LSTM num_layers: 1
    
    - OUT_SIZE: 15
    
    - HIDDEN_DIMENSION: 10
    
    - EPOCHS: 5
   
    - LEARNING RATE: 0.001
    
    - LOSS FUNCTION: NLLLOSS
    
    - OPTIMIZER: ADAM

### high score using unbalanced set (50+% FF)
- **TRAINING ACCURACY HIGH SCORE:** 57.8 (91% FF)

- **TEST ACCURACY HIGH SCORE:** 60.86 (88% FF) 2.5%+ naive 


- ****HYPERPARAMETERS:****

    - LSTM num_layers: 1
    
    - OUT_SIZE: 15
    
    - HIDDEN_DIMENSION: 10
    
    - EPOCHS: 20
   
    - LEARNING RATE: 0.001
    
    - LOSS FUNCTION: NLLLOSS
    
    - OPTIMIZER: ADAM